In [1]:
# Regularized Autoencoders

import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:

# We'll use the MNIST dataset

transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root="./data",
                         train=True,
                         download=True,
                         transform=transform)

test_dataset = datasets.MNIST(root="./data",
                              train=False,
                              download=True,
                              transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=32,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=32,
                                          shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 450504526.58it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 111851979.52it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 219768387.83it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 21747178.96it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [11]:
"""
transform = transforms.ToTensor()

train_data = datasets.CIFAR10(root="./data",
                              train=True,
                              download=True,
                              transform=transform)

test_data = datasets.CIFAR10(root="./data",
                             train=False,
                             download=True,
                             transform=transform)
"""

'\ntransform = transforms.ToTensor()\n\ntrain_data = datasets.CIFAR10(root="./data", \n                              train=True, \n                              download=True, \n                              transform=transform)\n\ntest_data = datasets.CIFAR10(root="./data", \n                             train=False, \n                             download=True, \n                             transform=transform)\n'

In [12]:
"""
batch_size = 128
num_workers = 0

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                           num_workers=num_workers, shuffle=False)

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
"""

"\nbatch_size = 128\nnum_workers = 0\n \ntrain_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, \n                                           num_workers=num_workers, shuffle=True)\ntest_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, \n                                           num_workers=num_workers, shuffle=False)\n\nclasses = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']\n"

In [18]:
class RegularizedAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True)
        )

        self.decoder = nn.Sequential(
            nn.Linear(32,64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 784),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [19]:
ae_model = RegularizedAE().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(ae_model.parameters(), lr=1e-3, weight_decay=1e-5)

In [23]:
def create_noise(images, noise_factor):
    return images * (1 - noise_factor) + torch.rand(images.size()).to(device) * noise_factor

In [ ]:
n_epochs = 10
noise_factor = 0.1

# We are artificially adding noise to the dataset and pass in to our AE.
for epoch in range(n_epochs):
    train_loss =  0.0
    for i, data in enumerate(train_loader):
        images, _ = data
        images = images.to(device)  # Move images to the device
        images = images.view(images.size(0), -1)
        noisy_imgs = create_noise(images, noise_factor)
        noisy_imag = noisy_imgs.to(device)
        optimizer.zero_grad()
        outputs = ae_model(noisy_imgs)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)

    train_loss = train_loss / len(train_loader.dataset)  # Correct calculation of average loss
    print(f"Epoch {epoch + 1} of {n_epochs}, Loss: {train_loss}")


Epoch 1 of 10, Loss: 0.01571075299580892
Epoch 2 of 10, Loss: 0.015378648275633653
Epoch 3 of 10, Loss: 0.015158126931885878
Epoch 4 of 10, Loss: 0.01497927515655756
Epoch 5 of 10, Loss: 0.01485109131783247
Epoch 6 of 10, Loss: 0.014717620947957038
Epoch 7 of 10, Loss: 0.014612542220950127
Epoch 8 of 10, Loss: 0.014525073391199112
Epoch 9 of 10, Loss: 0.014457494953771433
Epoch 10 of 10, Loss: 0.014388798725605011
